In [ ]:
%load_ext autoreload
%autoreload 1
%aimport game

import game
import random
import time

game._build_row_tables()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [159]:
def noskill(bs, action_space):
    return random.choice(action_space)(bs)

def naive_minimize_empty_tiles(bs, action_space):
    best_action, best_score = None, 0
    for action in action_space:
        new_bs = action(bs)
        num_empty_tiles = len(game.get_empty_tiles(new_bs))
        if num_empty_tiles > best_score:
            best_action, best_score = action, num_empty_tiles
    return best_action(bs)

In [165]:
g = game.Game()

results, ai_name = g.run_game(noskill, num_games=1000)
g.print_results(results, ai_name)
g.plot_results(results, ai_name)

results, ai_name = g.run_game(naive_minimize_empty_tiles, num_games=1000)
g.print_results(results, ai_name)
g.plot_results(results, ai_name)

NameError: name 'time' is not defined